Com base no arquivo dos campeões de formula 1 faça:

a) Incluir todos os registros em um base de dados.

b) Gerar um arquivo .xlsx com pandas com Nome do Piloto, equipe, Ano do titulo.formula1-1.txt

In [71]:
import mysql.connector
import pandas as pd

db = 'f1'
table = 'champ'
ddl = 'ano_do_titulo int, pais varchar(50), nome_do_piloto varchar(100), equipe varchar(100)'
sql = f'insert into {table} (ano_do_titulo, pais, nome_do_piloto, equipe) values (%s, %s, %s, %s)'

def sqlConnect(host, user, password, db = ''):
    return mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=db
)

mydb = sqlConnect('localhost', 'root', 'password')
mycursor = mydb.cursor()

mycursor.execute(f'show databases like "{db}"')
if len(mycursor.fetchall()):
    mycursor.execute(f'drop database {db}')
mycursor.execute(f'create database {db}')

mydb = sqlConnect('localhost', 'root', 'password', db)
mycursor = mydb.cursor()

mycursor.execute(f'show tables like "{table}"')
if len(mycursor.fetchall()):
    mycursor.execute(f'drop database {table}')
mycursor.execute(f'create table {table} ({ddl})')

with open('formula1-1.txt', encoding='UTF-8') as txt:
    readlines = txt.readlines()
    for line in readlines:
        line = line.split(';')
        line = tuple(map(lambda x: x.strip(), line))
        mycursor.execute(sql, line)
        mydb.commit()

mycursor.execute(f'select * from {table}')
xlsx = pd.DataFrame(mycursor.fetchall(), columns=['ano_do_titulo', 'pais', 'nome_do_piloto', 'equipe'])
xlsx[['nome_do_piloto', 'equipe', 'ano_do_titulo']].to_excel('campeao.xlsx', index=0)

mydb.close()